<a href="https://colab.research.google.com/github/shabnamsattar/OCEAN/blob/main/OCEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np

In [2]:
!git clone https://github.com/shabnamsattar/OCEAN.git
%cd OCEAN

Cloning into 'OCEAN'...
remote: Enumerating objects: 524, done.
remote: Counting objects: 100% (226/226), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 524 (delta 112), reused 39 (delta 39), pack-reused 298 (from 3)
Receiving objects: 100% (524/524), 15.60 MiB | 10.17 MiB/s, done.
Resolving deltas: 100% (180/180), done.
Updating files: 100% (218/218), done.
/content/OCEAN


In [3]:
# Gurobi licence
os.environ["GRB_LICENSE_FILE"] = "/content/gurobi.lic"

In [4]:
!pip install -q -r requirements.txt
!pip install -q gurobipy
!pip install -q -e .

ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement numpy==1.21.5 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 1.19.5, 1.20.0, 1.20.1, 1.20.2, 1.20.3, 1.21.0, 1.21.1, 1.22.0, 1.22.1, 1.22.2, 1.22.3, 1.22.4, 1.23.0, 1.23.1, 1.23.2, 1.23.3, 1.23

In [16]:
from sklearn.ensemble import IsolationForest
from src.DatasetReader import DatasetReader
from src.IfCounterfactual import IfCounterfactualMilp
from src.CounterFactualParameters import FeatureType, FeatureActionnability, TreeConstraintsType, BinaryDecisionVariables


reader = DatasetReader("datasets/wine.csv")
X = reader.X.values

# Train IF
ilf = IsolationForest(random_state=1, max_samples=min(256, len(X)), n_estimators=100, contamination=0.1)
ilf.fit(X)

# Pick an outlier x0 (taking one predicted as outlier by the model)
preds = ilf.predict(X)            # 1=inlier, -1=outlier
outlier_idx = int(np.where(preds == -1)[0][2])
x0 = [X[outlier_idx]]

# Building a Counterfactual(goal: become inlier)
milp = IfCounterfactualMilp(
    isolationForest=ilf,
    sample=x0,
    objectiveNorm=1,   #L1
    verbose=False,
    featuresType=reader.featuresType,
    featuresPossibleValues=reader.featuresPossibleValues,
    featuresActionnability=reader.featuresActionnability,
    oneHotEncoding=reader.oneHotEncoding,
    constraintsType=TreeConstraintsType.LinearCombinationOfPlanes,
    binaryDecisionVariables=BinaryDecisionVariables.PathFlow_y
)

milp.buildModel(decision_threshold=0.0)
cf = milp.solveModel(time_limit=300, threads=4)

print("Solved:", cf)
print("x0 predicted:", ilf.predict(np.array(x0))[0])  # expected -1
print("x' predicted:", ilf.predict(np.array(milp.x_sol))[0])  # expected  1


# show both original and counterfactual in original scale
x0_orig  = reader.format_explanation([x0[0]])      # decode + unscale x0
xcf_orig = reader.format_explanation(milp.x_sol)   # decode + unscale x'

print("Original (original scale):     ", x0_orig)
print("Counterfactual (original scale):", xcf_orig)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2689274
Academic license 2689274 - for non-commercial use only - registered to sh___@studenti.unipd.it
Solved: True
x0 predicted: -1
x' predicted: 1
Original (original scale):      [[1.42e+01 1.76e+00 2.45e+00 1.52e+01 1.12e+02 3.27e+00 3.39e+00 3.40e-01
  1.97e+00 6.75e+00 1.05e+00 2.85e+00 1.45e+03]]
Counterfactual (original scale): [[1.42000000e+01 1.76000000e+00 2.45000000e+00 1.68039499e+01
  1.10000000e+02 3.22454323e+00 3.33355460e+00 3.40000000e-01
  1.97000000e+00 6.63213313e+00 1.05359380e+00 2.85000000e+00
  1.45000000e+03]]


In [5]:
from sklearn.ensemble import IsolationForest
from src.DatasetReader import DatasetReader
from src.IfCounterfactual import IfCounterfactualMilp
from src.CounterFactualParameters import FeatureType, FeatureActionnability, TreeConstraintsType, BinaryDecisionVariables


reader = DatasetReader("datasets/cardio.csv")
X = reader.X.values

# Train IF
ilf = IsolationForest(random_state=1, max_samples=min(256, len(X)), n_estimators=100, contamination=0.1)
ilf.fit(X)

# Pick an outlier x0 (taking one predicted as outlier by the model)
preds = ilf.predict(X)            # 1=inlier, -1=outlier
outlier_idx = int(np.where(preds == -1)[0][0])
x0 = [X[outlier_idx]]

# Building a Counterfactual(goal: become inlier)
milp = IfCounterfactualMilp(
    isolationForest=ilf,
    sample=x0,
    objectiveNorm=1,   #L1
    verbose=False,
    featuresType=reader.featuresType,
    featuresPossibleValues=reader.featuresPossibleValues,
    featuresActionnability=reader.featuresActionnability,
    oneHotEncoding=reader.oneHotEncoding,
    constraintsType=TreeConstraintsType.LinearCombinationOfPlanes,
    binaryDecisionVariables=BinaryDecisionVariables.PathFlow_y
)

milp.buildModel(decision_threshold=0.0)
cf = milp.solveModel(time_limit=300, threads=4)

print("Solved:", cf)
print("x0 predicted:", ilf.predict(np.array(x0))[0])  # expected -1
print("x' predicted:", ilf.predict(np.array(milp.x_sol))[0])  # expected  1


# show both original and counterfactual in original scale
x0_orig  = reader.format_explanation([x0[0]])      # decode + unscale x0
xcf_orig = reader.format_explanation(milp.x_sol)   # decode + unscale x'

print("Original (original scale):     ", x0_orig)
print("Counterfactual (original scale):", xcf_orig)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2689274
Academic license 2689274 - for non-commercial use only - registered to sh___@studenti.unipd.it
Solved: True
x0 predicted: -1
x' predicted: 1
Original (original scale):      [[-0.20672176  0.55629255  8.3946632  -0.17740764  0.94292225 -0.06140064
   1.10441598 -1.41114256  0.99069677 -0.42048735 -0.0533886   0.87701669
  -0.83401339  0.53469408  0.9666375  -0.48827889  0.4216955  -0.50921552
  -0.11333832  1.8478356  -0.49329397]]
Counterfactual (original scale): [[-0.20672176  0.55629255  8.3946632  -0.17740764  0.94292225 -0.06140064
   1.10441598 -1.41114256  0.99069677 -0.42048735 -0.0533886   0.87701669
  -0.83401339  0.53469408  1.01086897 -0.48827889  0.34913677 -0.50921552
  -0.04691325  1.8478356  -0.49329397]]


In [8]:
ilf.score_samples(milp.x_sol)

array([-0.50390731])

In [9]:
ilf.decision_function(milp.x_sol)

array([4.79428808e-05])

In [10]:
ilf.score_samples(x0)

array([-0.51080724])

In [11]:
ilf.decision_function(x0)

array([-0.00685198])